# Investigation: Streaming Softmax

**From The Nature of Fast, Chapter 4: Chunking**

This notebook derives the streaming softmax algorithm step by step. You'll discover how associativity enables computing softmax without materializing the entire attention matrix.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ttsugriy/performance-book/blob/main/notebooks/tier2-experimental/04-streaming-softmax.ipynb)

---

## Setup

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

# For numerical stability
np.set_printoptions(precision=6, suppress=True)

print("Setup complete!")

## The Problem: Softmax Needs Everything

Standard softmax requires seeing all values to compute the denominator:

$$\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}$$

This means we can't compute any output until we've seen all inputs. Or can we?

In [ ]:
def naive_softmax(x):
    """Standard softmax - requires all values."""
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

# Test it
x = np.array([1.0, 2.0, 3.0, 4.0])
print(f"Input: {x}")
print(f"Softmax: {naive_softmax(x)}")
print(f"Sum (should be 1.0): {naive_softmax(x).sum():.10f}")

### Problem: Numerical Overflow

What happens with large values?

In [ ]:
# Large values cause overflow
x_large = np.array([1000.0, 1001.0, 1002.0, 1003.0])
print(f"Input: {x_large}")
print(f"exp(1000) = {np.exp(1000)}")
print(f"Naive softmax: {naive_softmax(x_large)}")
print("\n☠️ NaN! The naive approach overflows.")

## Step 1: The Max Trick (Stable Softmax)

Subtract the maximum value before exponentiating:

$$\text{softmax}(x_i) = \frac{e^{x_i - \max(x)}}{\sum_j e^{x_j - \max(x)}}$$

This is mathematically equivalent (the max cancels out) but numerically stable.

In [ ]:
def stable_softmax(x):
    """Numerically stable softmax using max trick."""
    x_max = np.max(x)
    exp_x = np.exp(x - x_max)  # Subtract max before exp
    return exp_x / np.sum(exp_x)

# Now it works!
print(f"Input: {x_large}")
print(f"Stable softmax: {stable_softmax(x_large)}")
print(f"Sum: {stable_softmax(x_large).sum():.10f}")

# Verify equivalence for normal values
print(f"\nNaive and stable match: {np.allclose(naive_softmax(x), stable_softmax(x))}")

## Step 2: The Streaming Challenge

But we still need to see all values to find the max!

**Question**: Can we compute softmax in chunks, without ever holding all values in memory?

Let's think about what we need:
1. The maximum value (to prevent overflow)
2. The sum of exponentials (for normalization)

Can we update these incrementally?

In [ ]:
# First attempt: Process in chunks, track running max and sum
def chunked_softmax_v1(x, chunk_size=2):
    """First attempt at chunked softmax."""
    n = len(x)
    
    # First pass: find max
    running_max = float('-inf')
    for i in range(0, n, chunk_size):
        chunk = x[i:i+chunk_size]
        chunk_max = np.max(chunk)
        running_max = max(running_max, chunk_max)
    
    # Second pass: compute exp and sum
    running_sum = 0
    for i in range(0, n, chunk_size):
        chunk = x[i:i+chunk_size]
        running_sum += np.sum(np.exp(chunk - running_max))
    
    # Third pass: compute output
    output = np.zeros(n)
    for i in range(0, n, chunk_size):
        chunk = x[i:i+chunk_size]
        output[i:i+chunk_size] = np.exp(chunk - running_max) / running_sum
    
    return output

# Test it
result = chunked_softmax_v1(x)
print(f"Chunked v1: {result}")
print(f"Matches stable: {np.allclose(result, stable_softmax(x))}")
print(f"\n⚠️ Problem: We needed THREE passes over the data!")

## Step 3: The Insight - One Pass Is Possible!

Here's the key insight: when we see a new maximum, we can **rescale** our running sum.

If we've computed $\sum e^{x_i - m_{old}}$ and find a new max $m_{new}$, we can convert:

$$\sum e^{x_i - m_{old}} \cdot e^{m_{old} - m_{new}} = \sum e^{x_i - m_{new}}$$

This lets us maintain a running sum while updating the max!

In [ ]:
def streaming_softmax_stats(x, chunk_size=2):
    """Compute softmax max and sum in ONE pass using rescaling."""
    n = len(x)
    
    # State: (running_max, running_sum)
    running_max = float('-inf')
    running_sum = 0.0
    
    for i in range(0, n, chunk_size):
        chunk = x[i:i+chunk_size]
        chunk_max = np.max(chunk)
        
        if chunk_max > running_max:
            # Rescale the running sum!
            running_sum = running_sum * np.exp(running_max - chunk_max)
            running_max = chunk_max
        
        # Add this chunk's contribution
        running_sum += np.sum(np.exp(chunk - running_max))
        
        print(f"After chunk {i//chunk_size + 1}: max={running_max:.2f}, sum={running_sum:.4f}")
    
    return running_max, running_sum

# Test on our example
print(f"Input: {x}")
final_max, final_sum = streaming_softmax_stats(x)
print(f"\nFinal: max={final_max}, sum={final_sum}")

# Verify against standard approach
expected_max = np.max(x)
expected_sum = np.sum(np.exp(x - expected_max))
print(f"Expected: max={expected_max}, sum={expected_sum}")
print(f"Match: {np.isclose(final_max, expected_max) and np.isclose(final_sum, expected_sum)}")

## Step 4: The Full Streaming Softmax

For attention, we don't just want softmax values—we want the **weighted sum** of values V.

We need to track: (max, sum, weighted_output)

When the max changes, we rescale both sum AND the accumulated output!

In [ ]:
def streaming_attention(scores, values, chunk_size=2):
    """
    Compute softmax(scores) @ values in a streaming fashion.
    
    This is the core of FlashAttention!
    
    Args:
        scores: [n] attention scores for one query
        values: [n, d] value vectors
        chunk_size: how many scores/values to process at once
    
    Returns:
        [d] output vector = softmax(scores) @ values
    """
    n = len(scores)
    d = values.shape[1]
    
    # State: (running_max, running_sum, running_output)
    running_max = float('-inf')
    running_sum = 0.0
    running_output = np.zeros(d)
    
    for i in range(0, n, chunk_size):
        chunk_scores = scores[i:i+chunk_size]
        chunk_values = values[i:i+chunk_size]
        chunk_max = np.max(chunk_scores)
        
        if chunk_max > running_max:
            # Rescale everything!
            scale = np.exp(running_max - chunk_max)
            running_sum = running_sum * scale
            running_output = running_output * scale
            running_max = chunk_max
        
        # Compute weights for this chunk
        chunk_weights = np.exp(chunk_scores - running_max)
        
        # Update running totals
        running_sum += np.sum(chunk_weights)
        running_output += chunk_weights @ chunk_values  # Weighted sum
    
    # Final normalization
    return running_output / running_sum

# Test it!
np.random.seed(42)
n, d = 8, 4
scores = np.random.randn(n)
values = np.random.randn(n, d)

# Streaming version
streaming_result = streaming_attention(scores, values, chunk_size=2)

# Standard version for comparison
weights = stable_softmax(scores)
standard_result = weights @ values

print(f"Streaming result: {streaming_result}")
print(f"Standard result:  {standard_result}")
print(f"\nMatch: {np.allclose(streaming_result, standard_result)}")
print(f"Max difference: {np.max(np.abs(streaming_result - standard_result)):.2e}")

## 🎉 We Did It!

We just derived the core algorithm of FlashAttention:

1. Process scores and values in chunks
2. Maintain running (max, sum, output) state
3. Rescale when max changes
4. Normalize at the end

This means we never need to store the full N×N attention matrix!

## Investigation: Memory Savings

Let's quantify how much memory this saves.

In [ ]:
def memory_comparison(seq_lengths):
    """Compare memory usage: standard vs streaming."""
    
    results = []
    for n in seq_lengths:
        # Standard attention: stores N×N matrix
        standard_memory = n * n * 4  # float32 = 4 bytes
        
        # Streaming: only stores chunk_size scores + running state
        chunk_size = 64  # Typical FlashAttention block size
        streaming_memory = chunk_size * 4 + 3 * 4  # chunk + (max, sum, one output float)
        
        results.append({
            'n': n,
            'standard_mb': standard_memory / 1e6,
            'streaming_mb': streaming_memory / 1e6,
            'ratio': standard_memory / streaming_memory,
        })
        
        print(f"N={n:>6}: Standard={standard_memory/1e6:>8.2f} MB, "
              f"Streaming={streaming_memory/1e6:>8.4f} MB, "
              f"Savings={standard_memory/streaming_memory:>8.0f}×")
    
    return results

seq_lengths = [512, 1024, 2048, 4096, 8192, 16384, 32768, 65536]
results = memory_comparison(seq_lengths)

In [ ]:
# Visualize
ns = [r['n'] for r in results]
standard = [r['standard_mb'] for r in results]
streaming = [r['streaming_mb'] for r in results]

plt.figure(figsize=(10, 6))
plt.loglog(ns, standard, 'o-', label='Standard (N² memory)', linewidth=2, markersize=8)
plt.loglog(ns, streaming, 's-', label='Streaming (constant memory)', linewidth=2, markersize=8)

# Add GPU memory limit
plt.axhline(y=16000, color='red', linestyle='--', alpha=0.5, label='16 GB GPU memory')

plt.xlabel('Sequence Length', fontsize=12)
plt.ylabel('Memory (MB)', fontsize=12)
plt.title('Attention Memory: Standard vs Streaming', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Insight: Standard attention runs out of memory around N=65K.")
print("   Streaming can handle millions of tokens!")

## The Associativity Connection

Why does this work? Because of **associativity**.

The key operation is combining two (max, sum, output) tuples:

```
combine((m1, s1, o1), (m2, s2, o2)) = (
    max(m1, m2),
    s1 * exp(m1 - new_max) + s2 * exp(m2 - new_max),
    o1 * exp(m1 - new_max) + o2 * exp(m2 - new_max)
)
```

This combination is **associative**: `combine(combine(a, b), c) = combine(a, combine(b, c))`

Which means we can chunk arbitrarily!

In [ ]:
def combine_states(state1, state2):
    """Combine two (max, sum, output) states."""
    m1, s1, o1 = state1
    m2, s2, o2 = state2
    
    new_max = max(m1, m2)
    scale1 = np.exp(m1 - new_max)
    scale2 = np.exp(m2 - new_max)
    
    new_sum = s1 * scale1 + s2 * scale2
    new_output = o1 * scale1 + o2 * scale2
    
    return (new_max, new_sum, new_output)

# Verify associativity
np.random.seed(42)
a = (np.random.randn(), np.random.rand() + 0.1, np.random.randn())
b = (np.random.randn(), np.random.rand() + 0.1, np.random.randn())
c = (np.random.randn(), np.random.rand() + 0.1, np.random.randn())

# (a ⊕ b) ⊕ c
left = combine_states(combine_states(a, b), c)

# a ⊕ (b ⊕ c)
right = combine_states(a, combine_states(b, c))

print(f"(a ⊕ b) ⊕ c = {left}")
print(f"a ⊕ (b ⊕ c) = {right}")
print(f"\nAssociative: {np.allclose(left, right)}")

## Your Turn: Experiments

1. **Different chunk sizes**: How does chunk size affect numerical accuracy?

2. **Parallel reduction**: Since `combine` is associative, we can parallelize! Implement a parallel version.

3. **Multi-query**: Extend to handle multiple queries (multiple rows of Q).

In [ ]:
# Your experiments here!

# Example: Test different chunk sizes
def test_chunk_sizes():
    np.random.seed(123)
    n, d = 1024, 64
    scores = np.random.randn(n) * 2  # Larger variance for more interesting behavior
    values = np.random.randn(n, d)
    
    # Ground truth
    weights = stable_softmax(scores)
    ground_truth = weights @ values
    
    for chunk_size in [1, 2, 4, 8, 16, 32, 64, 128, 256]:
        result = streaming_attention(scores, values, chunk_size)
        max_error = np.max(np.abs(result - ground_truth))
        print(f"Chunk size {chunk_size:>4}: max error = {max_error:.2e}")

test_chunk_sizes()
print("\n💡 Notice: All chunk sizes give the same answer (within floating point precision)!")

## Key Takeaways

1. **The max trick** prevents overflow by subtracting max before exp.

2. **Rescaling** lets us update a running sum when we find a new max.

3. **Streaming attention** maintains (max, sum, output) state and updates incrementally.

4. **Associativity** is what makes chunking possible—we can combine partial results in any order.

5. **Memory savings** are dramatic: O(N²) → O(1) for the attention matrix.

This is the mathematical foundation of FlashAttention!

---

*Continue to [Chapter 11: FlashAttention Investigation](https://ttsugriy.github.io/performance-book/chapters/11-flash-attention.html)*